In [18]:
R=FractionField(QQ['q','t'])
q=R.0
t=R.1
MaxDepth=65536
n=5
m=7
def WeylQuot(part):
    Quot=factorial(part.size())
    for i in range(part.length()):
        Quot=Quot//part[i]
    return Quot

##orb(A,B,d,nil,func) computes the orbital integral on gl_{s_i}, for one-step case s_i=n
##It depends on A=s_i/s_{i-1}, B=s_{i-1}, d=numerator of depth, in one-step case it is =m
##nil is a partition corresponding to a nilpotent orbit
##func is a binary vector determining which lattice we take the characteristic function of
##MaxDepth is used to make everything negative, with some work it can be avoided but Python3 floor division 
##behaves badly if we don't have it
def orb(A,B,d,nil,func):
    sum=0
    quot=WeylQuot(nil)
    k=0
    w=[0 for l in range(nil.size())]
    for i in range(nil.length()): 
        for j in range(nil[i]): 
            w[k]=i
            k+=1
    cosets=Arrangements(w,nil.size()).list()
    for coset in cosets:
        power=0
        for j in range(A*B):
            for k in range(A*B):
                if (coset[j]>=coset[k]): continue
                if ((j//B!=k//B) or (j%B<k%B)): 
                    power += int((d*(j//B-k//B)-MaxDepth*A)/A)
                else: 
                    power += (-MaxDepth+1)
                if((k//B-j//B)==1):
                    if(k-j>B or ((k-j==B) and func[j])):
                        power-=(-MaxDepth)
                    else:
                        power-=(-MaxDepth+1)
                elif((j//B-k//B)==(A-1)):
                    if((j-k)<=((A-1)*B) or ((j-k==(A-1)*B+1) and func[j-1])):
                        power-=(-MaxDepth+d)
                    else:
                        power-=(-MaxDepth+d+1)
                else: power-=int((d*(j//B-k//B+1)-MaxDepth*A)/A)
        sum+=q^(power)
    return sum
#orb2 computes the nilpotent orbital integrals on gl_{s_{i-1}}
def orb2(n,d,nil,func):
    sum=0
    quot
    k=0
    w=[0 for l in range(nil.size())]
    for i in range(nil.length()): 
        for j in range(nil[i]): 
            w[k]=i
            k+=1
    cosets=Arrangements(w,nil.size()).list()
    for coset in cosets:
        power=0
        for j in range(n):
            for k in range(n):
                if(coset[j]>=coset[k]): continue
                if(j-k!=1 or (not func[j-1])):
                    power-=d
                else:
                    power-=d-1
        sum+=q^(power)
    return sum

#below is the main function (currently ONLY implemented for torus knots!!!)
#we loop over test functions and nilpotent orbits to form a 
#2^{s_i-1} x part(s_i) matrix of nilpotent orbital integrals
#TODO: given a list of depths, compute the transition matrices for nilp orbital integrals
#and finally find Shalika germs in general case

a=1<<(n-1)
M=Matrix(R,a,Partitions(n).cardinality())
testfunc=[False for l in range(n-1)]
for j in range(a):
    bits=j
    for k in reversed(range(0,n-1)): 
        testfunc[k]=(bits%2==0)
        bits//=2
    p=0
    #print(testfunc)
    for part in Partitions(n):
        M[j,p]=orb(n,1,m,part,testfunc)
        p+=1
#print(M)
#now we solve the resulting linear system, vec stores Shalika germs
v=[0 for l in range(a)]
v[0]=1
vec=M.solve_right(v)
#print(vec)
#print normalized Shalika germs:
pp=0
for part in Partitions(n):
    print("The normalized Shalika germ for "+str(part)+" is")
    NormalizedSha=vec[pp]
    NormalizedSha=NormalizedSha*(1-q^(-1))^(n-1)*q^(-(n-1)*(m-1)/2)
    print(NormalizedSha)
    pp+=1
pp=0
Sph=0
for part in Partitions(n):
    prod=1
    prod=1
    for i in range (part.length()):
        for k in range(part[i]):
            prod=prod*sum(q^j for j in range(k+1))
#    print(factor(vec[pp]/prod))
    Sph=Sph+vec[pp]/prod
    pp+=1
Sph.reduce()
print("The weight polynomial for the spherical affine Springer fiber is")
print(Sph)

The normalized Shalika germ for [5] is
(-q^4 - q^3 - q^2 - q - 1)/(-q^16)
The normalized Shalika germ for [4, 1] is
(-q^3 - 2*q^2 - q - 1)/q^10
The normalized Shalika germ for [3, 2] is
(-q^3 - q^2 - 2*q - 1)/q^7
The normalized Shalika germ for [3, 1, 1] is
(q^2 + 2*q + 2)/q^5
The normalized Shalika germ for [2, 2, 1] is
(2*q^2 + 2*q + 1)/q^4
The normalized Shalika germ for [2, 1, 1, 1] is
(-3*q - 2)/q^2
The normalized Shalika germ for [1, 1, 1, 1, 1] is
1
The weight polynomial for the spherical affine Springer fiber is
q^12 + 4*q^11 + 7*q^10 + 9*q^9 + 10*q^8 + 9*q^7 + 8*q^6 + 6*q^5 + 5*q^4 + 3*q^3 + 2*q^2 + q + 1
